In [ ]:
### 只做单模型，其他的内容，等等再说，现在把单模型的表现提升上去再说吧
### 测试看下来，单纯得跑模型似乎会应该Y值的方差而产生巨大问题，我们需要重新做一次分类，明天的核心任务吧

In [1]:
import sys
sys.path.append("../")
import pandas as pd
import numpy as np
from dateutil.parser import parse
from sklearn import metrics
import datetime

import model_ml as mm
import feat_engineering as fe
import feat_selection as fs
from param_config import config

In [34]:
dfTrain = pd.read_csv(config.original_train_data_path)
dfPred = pd.read_csv(config.original_pred_data_path)
predictors = dfPred.columns.tolist()[4:]
check_missing = ['PartI_1','PartII_1','PartIII_1','PartIV_1','PartV_1','PartVI_1']


###清理异常Y值
dfTrain = dfTrain.loc[dfTrain['Y']<dfTrain['Y'].max()]
dfTrain = dfTrain.reset_index(drop=True)

#dfTrain = dfTrain.loc[dfTrain['Y']>dfTrain['Y'].quantile(0.9)]

In [35]:
def FeatAll(train,pred):
    dfAll = pd.concat([train,pred])
    dfAll = dfAll.reset_index(drop=True)
    dfAll['date'] = (pd.to_datetime(dfAll['date']) - parse('2017-10-09')).dt.days
    
    ###尝试处理异常值
    for var in predictors:
        dfAll.loc[dfAll[var]>=dfAll[var].quantile(0.99)*3,var] = dfAll[var].quantile(0.99)*3
        #dfAll.loc[dfAll[var]>=df[var].quantile(0.99)*1.5,var] = np.nan

    ###这个部分将每一列分箱，并且计算基于该列分类的其他列的百分比顺序，目前看来没啥用
    var_dict = {}
    for colBase in predictors+['sex','age']:
        if dfAll[colBase].nunique()<=5:
            var_dict[colBase] = [x for x in predictors if x!=colBase]
        else:
            dfAll[colBase+'_based'] = pd.cut(dfAll[colBase],bins=5,labels=[colBase+'_bin_%d'%i for i in range(5)])
            dfAll[colBase+'_based'] = dfAll[colBase+'_based'].astype(str)
            dfAll[colBase+'_based'].fillna(colBase+'_bin_NA',inplace = True)
            var_dict[colBase+'_based'] = [x for x in predictors if x!=colBase]
    #dfAllPcentByOther = fe.pcent_by_other_col(dfAll,var_dict,['ID']) 
    #dfAllPcentByOther = pd.read_csv('../../Cache/pcent_by_other.csv')
    
    for var in dfAll.columns:
        if '_based' in var:
            del dfAll[var]
            
    ###根据日期放置一个百分比顺序作为尝试
    dfAllPcentByDate = fe.pcent_by_other_col(dfAll,{'date':predictors},['ID']) 
    
    '''dfAll['date'] = pd.qcut(dfAll['date'],q=10,labels=['date_bin_%d'%i for i in range(10)])
    dfAll = pd.concat([dfAll,pd.get_dummies(dfAll['date'])],axis=1)'''
    ###目前不想使用日期，觉得用处不大
    del dfAll['date']

    #for minus in
    
    for plus in [['PartII_1','PartII_2'],['PartIII_1','PartIII_2','PartIII_3']]:
        tmpVar =plus[0]
        tmpValue = dfAll[plus[0]].values
        for var in plus[1:]:
            tmpVar = tmpVar +'_plus_' + var
            tmpValue = tmpValue + dfAll[var].values
        dfAll[tmpVar] = tmpValue
    
    dfAll['PartI_9'] = dfAll['PartI_5']-dfAll['PartI_6']-dfAll['PartI_7']
    dfAll['PartII_5'] = dfAll['PartII_2']-dfAll['PartII_3']-dfAll['PartII_4']
    
    for ratio in [['PartII_1','PartII_2'],['PartI_6','PartI_5'],['PartI_7','PartI_5'],['PartII_3','PartII_2'],['PartII_4','PartII_2'],['PartV_2','PartV_1'],['PartII_1','PartII_2']]:
        dfAll[ratio[0]+'_divided_'+ratio[1]] = dfAll[ratio[0]]/dfAll[ratio[1]]
        
    for multiply in [['PartI_1','PartI_5'],['PartI_2','PartI_5'],['PartI_3','PartI_5'],['PartI_4','PartI_5'],['PartVI_1','PartVI_2'],['PartV_2','PartV_5'],['PartV_2','PartV_7'],['PartV_1','PartV_9'],['PartV_1','PartV_10'],['PartV_1','PartV_11'],['PartV_1','PartV_12'],['PartV_1','PartV_13']]:
        dfAll[multiply[0]+'_multiply_'+multiply[1]] = dfAll[multiply[0]]*dfAll[multiply[1]]
    
    
    ###看看missing 的情况
    dfAll['missing_cnt'] = dfAll[check_missing].isnull().sum(axis=1)
    for i in range(len(check_missing)):
        dfAll['missing_part%d'%i] = dfAll[check_missing[i]].isnull().astype(int)
    
    
    '''for key,group in {'PartI':['PartI_1','PartI_2','PartI_3','PartI_4'],'PartII':['PartII_1','PartII_3','PartII_4']}.items():
        dfAll['group_'+key+'_std'] = dfAll[group].std(axis=1)
        dfAll['group_'+key+'_mean'] = dfAll[group].mean(axis=1)
        dfAll['group_'+key+'_median'] = dfAll[group].median(axis=1)'''
    
    #dfAll = dfAll.merge(dfAllPcentByOther,'inner','ID')
    dfAll = dfAll.merge(dfAllPcentByDate,'inner','ID')
    
    
    dfTrain = dfAll.loc[dfAll['ID'].isin(train['ID'])]
    dfPred = dfAll.loc[dfAll['ID'].isin(pred['ID'])]
    
    return dfTrain,dfPred,var_dict

In [36]:
dfTrain,dfPred,var_dict = FeatAll(dfTrain,dfPred)
predictors = dfPred.columns.tolist()
predictors.remove('ID')
predictors.remove('Y')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [37]:
len(predictors)

105

In [50]:
np.corrco(dfTrain['missing_cnt'],dfTrain['Y'])

AttributeError: module 'numpy' has no attribute 'corr'

In [38]:
dfTrain =dfTrain.fillna(dfTrain.mean())

In [46]:
n_splits=5
early_stop=50
params = {'max_depth':5, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':1,'subsample':0.8,'colsample_bytree':0.8}
test_result,result,imp = mm.xgb_kfold(dfTrain,dfPred,predictors,n_splits,early_stop=early_stop,params=params)

No metric defined, will use rmse in the model
[0]	train-rmse:5.27704	eval-rmse:5.32367
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:2.24015	eval-rmse:2.39454
[200]	train-rmse:1.2928	eval-rmse:1.59734
[300]	train-rmse:1.03192	eval-rmse:1.4406
[400]	train-rmse:0.928532	eval-rmse:1.41445
[500]	train-rmse:0.868266	eval-rmse:1.40831
[600]	train-rmse:0.823516	eval-rmse:1.40452
[700]	train-rmse:0.788325	eval-rmse:1.40307
Stopping. Best iteration:
[656]	train-rmse:0.803753	eval-rmse:1.40235

Best tree is 657, performance is 0.803753, 1.402349
No metric defined, will use rmse in the model
[0]	train-rmse:5.29706	eval-rmse:5.24312
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:2.26268	eval-rmse:2.26035
[200]	train-rmse:1.31362	eval-rmse:1.43536
[300]	train-rmse:1.05237	eva

In [47]:
for i in range(1,n_splits+1):
    imp['imp_fold%d'%i] = imp['imp_fold%d'%i]/imp['imp_fold%d'%i].sum()
imp['sum_imp'] = imp[['imp_fold%d'%i for i in range(1,n_splits+1)]].sum(axis=1)

In [48]:

tmpPredictor = imp.sort_values('sum_imp',ascending=False)['variable'].values.tolist()[0:45]
imp.sort_values('sum_imp',ascending=False)

,lk,variable,imp_fold1,imp_fold2,imp_fold3,imp_fold4,imp_fold5,sum_imp
0,f0,PartIII_1,0.062831,0.063772,0.071151,0.067010,0.064298,0.329061
37,f37,age,0.047199,0.053276,0.051553,0.041144,0.045718,0.238890
2,f2,PartIII_3,0.033635,0.027910,0.034073,0.035295,0.036183,0.167096
1,f1,PartIII_2,0.031081,0.024094,0.031547,0.030253,0.025752,0.142725
3,f3,PartII_1,0.021106,0.023855,0.022328,0.024303,0.030152,0.121744
12,f12,PartI_1,0.021957,0.028149,0.025196,0.015378,0.025915,0.116596
95,f95,pcent_date_PartV_8,0.016848,0.019959,0.020280,0.015076,0.016788,0.088950
55,f55,PartV_2_multiply_PartV_7,0.015206,0.018368,0.020280,0.015983,0.017358,0.087196
5,f5,PartII_3,0.013077,0.020118,0.016251,0.017647,0.019803,0.086896
14,f14,PartI_3,0.018734,0.019084,0.017412,0.016941,0.014506,0.086677


In [49]:
test_result,result,imp = mm.xgb_kfold(dfTrain,dfPred,tmpPredictor,n_splits,early_stop=early_stop,params=params)

No metric defined, will use rmse in the model
[0]	train-rmse:5.27704	eval-rmse:5.32395
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:2.24308	eval-rmse:2.39208
[200]	train-rmse:1.30528	eval-rmse:1.59768
[300]	train-rmse:1.04576	eval-rmse:1.4414
[400]	train-rmse:0.95067	eval-rmse:1.41145
[500]	train-rmse:0.895278	eval-rmse:1.4032
[600]	train-rmse:0.852051	eval-rmse:1.39969
[700]	train-rmse:0.81785	eval-rmse:1.39802
Stopping. Best iteration:
[742]	train-rmse:0.805123	eval-rmse:1.39655

Best tree is 743, performance is 0.805123, 1.396553
No metric defined, will use rmse in the model
[0]	train-rmse:5.29706	eval-rmse:5.24338
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:2.26599	eval-rmse:2.25967
[200]	train-rmse:1.32564	eval-rmse:1.43448
[300]	train-rmse:1.06496	eval-

In [ ]:
###基础变量
1.83977998784

###增加部分比例数据，已经缺失体检项目情况
1.84295123211  imp来看，missing信息没有意义

###上一版去除missing
1.83839589522



###增加一些比值信息
1.84354856365


###根据日期的百分比，效果更加不好了
1.85046680299  变量选择之后 1.83862284729



###前面所有的特征
1.86013896413   变量选择之后  1.85446185976
更改了一下模型的参数 1.81447588639  变量选择之后  1.80873726211
用中位数填补缺失值 1.81136029536  变量选择之后  1.80584671605
将异常值取空 然后按中位数填补 1.83149451674  变量选择之后  1.82105909448
将异常值cap 然后按中位数填补 1.80727635099  变量选择之后  1.81587223984
根据上一版用均值填补缺失（已有cap）  1.81374808226 ///   1.8175380386


###增加更多特征，主要涉及加减乘除
1.81045571166 ///  1.78855151825

In [ ]:
n_splits=5
other_note ='_more_var'
result['score']=result[['Score_%d'%i for i in range(1,n_splits+1)]].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
result.to_csv('../../Submission/result/result_%s'%today+other_note+'.csv',index=False)
submit['score'].to_csv('../../Submission/submit_%s'%today+other_note+'.csv',header=False,index=False)
test_result.to_csv('../../Submission/test/test_result_%s'%today+other_note+'.csv',index=False)
imp.to_csv('../../Submission/imp/importance_%s'%today+other_note+'.csv',index=False)